In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.append(project_root)

In [2]:
import matplotlib.pyplot as plt

import torch
import torch.optim as optim

from src.models.cgcnn import CGCNN
from src.data.loader import get_loaders
from src.train.trainer import Trainer

In [3]:
train_loader, val_loader, test_loader = get_loaders("../data/processed",
                                                    target="formation_energy_per_atom",
                                                    batch_size=16,
                                                    num_workers=0,
                                                    train_ratio=0.8,
                                                    val_ratio=0.1,
                                                    seed=42)

model = CGCNN(
    node_fea_len=118,
    edge_fea_len=1,
    hidden_fea_len=16,
    n_conv=5,
    out_dim=1
)

optimizer = optim.Adam(model.parameters(), lr=1e-5)

trainer = Trainer(
    model,
    train_loader,
    val_loader,
    optimizer,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

In [ ]:
best_model, train_losses, val_maes = trainer.train(num_epochs=500, max_prints=20)

Epoch 1 [Train]:   2%|▏         | 25/1414 [00:01<00:43, 31.85it/s, loss=0.68] 

In [ ]:
plt.figure()
plt.plot(train_losses, label="train loss")
plt.plot(val_maes, label='val loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
model.load_state_dict(best_model)
trainer.test(test_loader, metric="mae")
trainer.test(test_loader, metric="rmse")